In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Computers.csv')
df.drop('Unnamed: 0',axis = 1, inplace = True)
df.head(2)

,price,speed,hd,ram,screen,cd,multi,premium,ads,trend
0,1499,25,80,4,14,no,no,yes,94,1
1,1795,33,85,2,14,no,no,yes,94,1


In [3]:
df.describe()

,price,speed,hd,ram,screen,ads,trend
count,6259.000000,6259.000000,6259.000000,6259.000000,6259.000000,6259.000000,6259.000000
mean,2219.576610,52.011024,416.601694,8.286947,14.608723,221.301007,15.926985
std,580.803956,21.157735,258.548445,5.631099,0.905115,74.835284,7.873984
min,949.000000,25.000000,80.000000,2.000000,14.000000,39.000000,1.000000
25%,1794.000000,33.000000,214.000000,4.000000,14.000000,162.500000,10.000000
50%,2144.000000,50.000000,340.000000,8.000000,14.000000,246.000000,16.000000
75%,2595.000000,66.000000,528.000000,8.000000,15.000000,275.000000,21.500000
max,5399.000000,100.000000,2100.000000,32.000000,17.000000,339.000000,35.000000


In [4]:
df.replace('no', '0', inplace= True)
df.replace('yes', '1', inplace= True)

In [5]:
df.head(2)

,price,speed,hd,ram,screen,cd,multi,premium,ads,trend
0,1499,25,80,4,14,0,0,1,94,1
1,1795,33,85,2,14,0,0,1,94,1


In [19]:
df.columns

Index(['price', 'speed', 'hd', 'ram', 'screen', 'cd', 'multi', 'premium',
       'ads', 'trend'],
      dtype='object')

In [6]:
X = df.drop('price', axis = 1).values
y = df['price'].values

In [7]:
sc = pp.StandardScaler()
X = sc.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [9]:
import xgboost as xgb

In [10]:
reg = xgb.XGBRegressor(objective='reg:squarederror')

In [11]:
reg.fit(X_train, y_train, eval_metric= 'mae')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
y_pred = reg.predict(X_test)

In [13]:
print(metrics.mean_absolute_error(y_test, y_pred))
metrics.mean_absolute_percentage_error(y_test, y_pred)

107.07474518634768


0.04973675287108478

In [63]:
param_grid = {
    'learning_rate': [0.2,0.1,0.3],
    'colsample_bytree': [1, 0.9,0.8],
    'max_depth': [4,5,6],
    'n_estimators': [50,100,150],
    'reg_lambda': [0.1,0.5,1],
    'subsample': [1, 0.9,0.8]
}

In [64]:
from sklearn.model_selection import GridSearchCV

In [65]:
param_search = GridSearchCV(estimator = xgb.XGBRegressor(objective='reg:squarederror', seed = 42),
                            param_grid = param_grid, cv=5, n_jobs= 10, scoring = 'neg_mean_absolute_error')

In [66]:
param_search.fit(X_train, y_train, eval_metric = 'mae')

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=...
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, seed=42,
                                    subsample=None, tree_method=None,
      

In [67]:
param_search.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 6,
 'n_estimators': 150,
 'reg_lambda': 0.1,
 'subsample': 0.9}

In [14]:
snd_reg = xgb.XGBRegressor(objective='reg:squarederror', seed = 42, colsample_bytree = 0.8, learning_rate= 0.1, max_depth = 6, 
                n_estimators = 150, reg_lambda = 0.1, subsample = 0.9)

In [15]:
snd_reg.fit(X_train, y_train, eval_metric = 'mae')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=150, n_jobs=4, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=0.1, scale_pos_weight=1, seed=42,
             subsample=0.9, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [17]:
print(metrics.mean_absolute_error(y_test, snd_reg.predict(X_test)))
metrics.mean_absolute_percentage_error(y_test, snd_reg.predict(X_test))   

106.47285788051617


0.04958317461626279

In [18]:
import pickle
with open("xgboost_computer_price.pkl", "wb") as h:
    pickle.dump(snd_reg, h)